## Setup

In [1]:
# import statements
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wrf
import dask as da

In [2]:
# function because some of the simulations have a messed-up calendar
def preprocess(ds_in: xr.Dataset) -> xr.Dataset:
    ds_in.XTIME.attrs["units"] = "minutes since 2021-09-01 20:00:00"
    ds_in.XTIME.attrs["description"] = "minutes since 2021-09-01 20:00:00"
    return(ds_in)

def load(init_time: str, ens_num: int):
    path = "/mnt/drive2/wof-runs/"+init_time+"/wrfwof*.{:02d}".format(ens_num)
    return(xr.open_mfdataset(path,preprocess=preprocess,
                          decode_times=False,combine="nested",
                          concat_dim="Time")
    )
def fix_xtime(ds_in: xr.Dataset,init_time: str) -> xr.Dataset:
    if init_time in ["20", "21", "22", "23"]:
        start_date = "2021-09-01"
    elif init_time in ["00", "01", "02"]:
        start_date = "2021-09-02"
    else:
        raise ValueError("Unexpected init_time provided.")
    ds_in.XTIME.attrs["units"] = "minutes since "+start_date+" "+init_time+":00:00"
    ds_in.XTIME.attrs["description"] = ("minutes since "+start_date+" "+
                                        init_time+":00:00")
    return(xr.decode_cf(ds_in))

In [3]:
print("21Z")
ds_21Z = xr.concat([fix_xtime(load("21Z",e),"21") for e in range(1,19)],
                       "ens")
# print("22Z")
# ds_22Z = xr.concat([fix_xtime(load("22Z",e),"22") for e in range(1,19)],
#                        "ens")
# print("23Z")
# ds_23Z = xr.concat([fix_xtime(load("23Z",e),"23") for e in range(1,19)],
#                        "ens")
# print("00Z")
# ds_00Z = xr.concat([fix_xtime(load("00Z",e),"00") for e in range(1,19)],
#                        "ens")

21Z


In [25]:
geo_staggered = ds_21Z.PH + ds_21Z.PHB
geo = 0.5 * (geo_staggered.isel(
    bottom_top_stag=slice(None,-1)
    ) + geo_staggered.isel(
    bottom_top_stag=slice(1,None)
    )
)
geo = geo.rename({'bottom_top_stag':
                  'bottom_top'})
geo['bottom_top'] = ds_21Z.bottom_top

In [28]:
wrf.slp(geo,ds_21Z.T,ds_21Z.P+ds_21Z.PB,ds_21Z.QVAPOR)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3be94ec790>>
Traceback (most recent call last):
  File "/home/janoski/miniforge3/envs/wof-3.11/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [27]:
ds_21Z

<xarray.Dataset> Size: 764GB
Dimensions:              (ens: 18, Time: 73, south_north: 300, west_east: 300,
                          bottom_top: 50, bottom_top_stag: 51,
                          soil_layers_stag: 9, west_east_stag: 301,
                          south_north_stag: 301)
Coordinates:
    XLAT                 (Time, south_north, west_east) float32 26MB 36.37 .....
    XLONG                (Time, south_north, west_east) float32 26MB -80.56 ....
    XTIME                (Time) datetime64[ns] 584B 2021-09-01T21:00:00 ... 2...
Dimensions without coordinates: ens, Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag
Data variables: (12/79)
    ALBEDO               (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    CLDFRA               (ens, Time, bottom_top, south_north, west_east) float32 24GB dask.array<chunksize=(1, 1, 50, 150, 150), meta=np.ndarray>
    COMPOSITE_REFL_10CM  (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    COSALPHA             (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    COSZEN               (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    EMISS                (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    ...                   ...
    VT_DBZ_WT            (ens, Time, bottom_top, south_north, west_east) float32 24GB dask.array<chunksize=(1, 1, 50, 150, 150), meta=np.ndarray>
    W                    (ens, Time, bottom_top_stag, south_north, west_east) float32 24GB dask.array<chunksize=(1, 1, 51, 150, 150), meta=np.ndarray>
    WSPD10MAX            (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    WSPD80               (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    W_UP_MAX             (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
    XLAND                (ens, Time, south_north, west_east) float32 473MB dask.array<chunksize=(1, 1, 150, 150), meta=np.ndarray>
Attributes: (12/74)
    TITLE:                            OUTPUT FROM WRF V3.9pre#2 MODEL
    START_DATE:                      2021-09-01_21:00:00
    WEST-EAST_GRID_DIMENSION:        301
    SOUTH-NORTH_GRID_DIMENSION:      301
    BOTTOM-TOP_GRID_DIMENSION:       51
    DX:                              3000.0
    ...                              ...
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      -1
    ETAC:                            0.0
    history:                         Tue Sep 21 19:19:36 2021: /scratch/softw...
    NCO:                             4.6.7